In [1]:
import pandas as pd

In [2]:
climate_data_2003 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2003_A_31-12-2003.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2003.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2003-01-01,00:00,0,"886,9",887,"886,5",-9999,"20,3","18,8","20,4","20,2","18,9","18,7",91,90,91,308,"2,1","1,4",NaN
1,2003-01-01,01:00,0,"887,3","887,3","886,9",-9999,"19,9","18,5","20,3","19,9","18,8","18,5",92,91,92,302,"2,2","1,3",NaN
2,2003-01-01,02:00,0,"887,8","887,8","887,3",-9999,"19,8","18,5","20,1","19,8","18,8","18,4",92,92,92,302,"2,7","1,6",NaN
3,2003-01-01,03:00,0,"886,9","887,8","886,9",-9999,"19,5","18,5","19,8","19,5","18,6","18,4",94,92,94,295,"3,4",2,NaN
4,2003-01-01,04:00,0,"886,8","886,9","886,7",-9999,"19,2",18,"19,5","19,2","18,5",18,94,93,93,313,"3,9","2,5",NaN


In [3]:
climate_data_2003['DATA (YYYY-MM-DD)'] = climate_data_2003['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2003['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2003['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2003['DATA (YYYY-MM-DD)'].head(), climate_data_2003['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2003.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2003.columns:
    if col != "Data":
        climate_data_2003[col] = (
            climate_data_2003[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2003[col] = pd.to_numeric(climate_data_2003[col], errors="coerce")


for col in climate_data_2003.columns:
    if col != "Data" and climate_data_2003[col].dtype == object:
        climate_data_2003[col] = (
            climate_data_2003[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2003[col] = pd.to_numeric(climate_data_2003[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2003 = climate_data_2003.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2003.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2003.head()

0   2003-01-01
1   2003-01-01
2   2003-01-01
3   2003-01-01
4   2003-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2003-01-01,5.4,1123.000000,21.091667,26.0,18.8,18.250000,84.500000,10.1,2.395833
1,2003-01-02,0.6,1652.214286,22.233333,27.2,18.9,18.033333,78.583333,10.9,2.741667
2,2003-01-03,8.4,1365.285714,21.658333,26.6,18.3,18.266667,81.708333,11.6,3.562500
3,2003-01-04,25.4,1423.785714,21.350000,26.8,17.5,17.716667,81.250000,13.9,4.216667
4,2003-01-05,0.8,1469.846154,21.879167,27.1,17.4,17.162500,75.875000,10.8,4.137500


In [4]:
climate_data_2004 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2004_A_31-12-2004.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2004.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2004-01-01,00:00,0,"886,8","886,8","886,3",-9999,"21,8","16,6","21,8","21,5","17,5","16,6",77,72,72,87,"4,9",",8",NaN
1,2004-01-01,01:00,0,"887,3","887,3","886,8",-9999,"20,7","17,2","21,8","20,7","17,8","16,7",82,72,80,274,"2,2",1,NaN
2,2004-01-01,02:00,0,"887,1","887,5","887,1",-9999,"21,9","17,2","21,9","20,8","17,8","17,1",82,75,75,30,"4,6","1,9",NaN
3,2004-01-01,03:00,0,"886,8","887,2","886,8",-9999,"21,2","17,7","21,9","21,1","17,9","17,1",81,74,80,303,"3,2","1,1",NaN
4,2004-01-01,04:00,0,"885,9","886,7","885,9",-9999,"21,2","17,8","21,2",21,"17,8","17,7",82,81,81,318,"2,7","1,1",NaN


In [5]:
climate_data_2004['DATA (YYYY-MM-DD)'] = climate_data_2004['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2004['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2004['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2004['DATA (YYYY-MM-DD)'].head(), climate_data_2004['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2004.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2004.columns:
    if col != "Data":
        climate_data_2004[col] = (
            climate_data_2004[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2004[col] = pd.to_numeric(climate_data_2004[col], errors="coerce")


for col in climate_data_2004.columns:
    if col != "Data" and climate_data_2004[col].dtype == object:
        climate_data_2004[col] = (
            climate_data_2004[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2004[col] = pd.to_numeric(climate_data_2004[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2004 = climate_data_2004.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2004.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2004.head()

0   2004-01-01
1   2004-01-01
2   2004-01-01
3   2004-01-01
4   2004-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2004-01-01,2.8,1092.692308,22.029167,26.9,19.9,17.908333,77.791667,8.1,1.837500
1,2004-01-02,0.8,1765.846154,22.579167,27.4,19.0,17.787500,75.958333,10.0,2.687500
2,2004-01-03,17.4,950.384615,22.029167,26.4,18.8,18.554167,81.458333,7.4,1.458333
3,2004-01-04,3.2,1178.857143,21.395833,26.3,18.8,17.975000,82.125000,7.6,1.891667
4,2004-01-05,0.0,1498.750000,21.917391,27.3,18.3,17.643478,77.391304,9.2,2.017391


In [6]:
climate_data_2005 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2005_A_31-12-2005.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2005.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2005-01-01,00:00,0,"886,1","886,2","885,5",-9999,"20,4","17,4","20,7","20,4","17,7","17,3",84,82,83,138,"3,2","1,8",NaN
1,2005-01-01,01:00,0,"887,2","887,2","886,1",-9999,"19,9","17,5","20,6","19,9","17,9","17,4",86,82,86,178,"5,3","2,3",NaN
2,2005-01-01,02:00,0,"887,6","887,7","887,2",-9999,"19,9","18,1","19,9","19,6","18,1","17,6",89,86,89,194,4,"1,6",NaN
3,2005-01-01,03:00,0,"887,3","887,7","887,3",-9999,"19,6","18,1",20,"19,6","18,2","18,1",91,89,91,171,"3,5","1,2",NaN
4,2005-01-01,04:00,0,"886,6","887,3","886,6",-9999,"19,2",18,"19,6","19,2","18,1","17,9",93,91,93,182,"2,1",0,NaN


In [7]:
climate_data_2005['DATA (YYYY-MM-DD)'] = climate_data_2005['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2005['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2005['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2005['DATA (YYYY-MM-DD)'].head(), climate_data_2005['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2005.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2005.columns:
    if col != "Data":
        climate_data_2005[col] = (
            climate_data_2005[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2005[col] = pd.to_numeric(climate_data_2005[col], errors="coerce")


for col in climate_data_2005.columns:
    if col != "Data" and climate_data_2005[col].dtype == object:
        climate_data_2005[col] = (
            climate_data_2005[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2005[col] = pd.to_numeric(climate_data_2005[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2005 = climate_data_2005.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2005.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2005.head()

0   2005-01-01
1   2005-01-01
2   2005-01-01
3   2005-01-01
4   2005-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2005-01-01,0.4,1728.400000,21.204167,26.0,19.1,17.829167,81.541667,7.1,1.533333
1,2005-01-02,8.8,1289.333333,20.800000,26.9,18.3,17.670833,83.208333,8.8,1.979167
2,2005-01-03,10.8,1229.200000,19.833333,24.1,17.8,17.287500,85.875000,11.5,3.091667
3,2005-01-04,16.2,1370.000000,20.595833,27.5,17.4,17.666667,84.333333,10.8,2.154167
4,2005-01-05,15.8,899.000000,20.804167,25.7,17.5,17.525000,82.583333,8.1,2.116667


In [8]:
climate_data_2006 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2006_A_31-12-2006.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2006.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2006-01-01,00:00,0,"885,7","885,7","885,5",-9999,"18,6","16,4","18,6","17,8","17,1","16,4",94,87,87,74,"2,2","1,4",NaN
1,2006-01-01,01:00,0,"886,2","886,2","885,7",-9999,"17,5","16,2","18,7","17,3","16,6","15,7",92,87,92,286,"2,7","1,2",NaN
2,2006-01-01,02:00,",2","886,4","886,4","886,2",-9999,"17,4","16,5","17,5","17,2","16,6","16,1",95,92,94,26,"1,9",0,NaN
3,2006-01-01,03:00,0,"886,3","886,4","886,2",-9999,"17,1","15,7","17,9","17,1","16,6","15,6",94,89,91,8,"2,2",0,NaN
4,2006-01-01,04:00,0,886,"886,4",886,-9999,"17,5","16,3","17,5","16,9","16,4","15,6",93,91,93,336,"1,2",",3",NaN


In [9]:
climate_data_2006['DATA (YYYY-MM-DD)'] = climate_data_2006['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2006['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2006['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2006['DATA (YYYY-MM-DD)'].head(), climate_data_2006['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2006.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2006.columns:
    if col != "Data":
        climate_data_2006[col] = (
            climate_data_2006[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2006[col] = pd.to_numeric(climate_data_2006[col], errors="coerce")


for col in climate_data_2006.columns:
    if col != "Data" and climate_data_2006[col].dtype == object:
        climate_data_2006[col] = (
            climate_data_2006[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2006[col] = pd.to_numeric(climate_data_2006[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2006 = climate_data_2006.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2006.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2006.head()

0   2006-01-01
1   2006-01-01
2   2006-01-01
3   2006-01-01
4   2006-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2006-01-01,0.2,1522.092857,20.379167,25.8,16.9,15.929167,76.500000,10.8,2.362500
1,2006-01-02,4.8,1393.135714,21.291667,27.0,18.0,17.008333,77.916667,9.0,2.462500
2,2006-01-03,9.6,941.628571,19.950000,26.8,18.0,17.141667,84.583333,16.3,1.979167
3,2006-01-04,0.2,1482.007143,20.866667,26.6,17.2,16.745833,79.083333,10.5,1.570833
4,2006-01-05,20.8,864.871429,20.250000,25.8,17.1,17.333333,83.875000,11.8,1.320833


In [10]:
climate_data_2007 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2007_A_31-12-2007.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2007.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2007-01-01,00:00,"6,8","887,1","887,1","886,6",-9999,"20,7","19,6","21,1","19,7","19,8","18,4",94,92,93,35,"6,3",3,NaN
1,2007-01-01,01:00,"1,6","887,4","887,5",887,-9999,"20,9","19,5",21,"20,6","19,7","19,4",94,91,92,21,"5,3","1,5",NaN
2,2007-01-01,02:00,0,"887,9",888,"887,4",-9999,"20,7","19,3","20,9","20,6","19,5","19,3",93,91,91,21,"3,3","1,3",NaN
3,2007-01-01,03:00,0,"887,7",888,"887,7",-9999,"19,5","17,5","20,8","19,5","19,3","17,5",92,88,89,297,"4,3",2,NaN
4,2007-01-01,04:00,7,"887,5","887,8","887,4",-9999,"18,1","17,2","19,5","18,1",18,"17,2",95,89,95,281,"10,1","3,5",NaN


In [11]:
climate_data_2007['DATA (YYYY-MM-DD)'] = climate_data_2007['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2007['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2007['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2007['DATA (YYYY-MM-DD)'].head(), climate_data_2007['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2007.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2007.columns:
    if col != "Data":
        climate_data_2007[col] = (
            climate_data_2007[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2007[col] = pd.to_numeric(climate_data_2007[col], errors="coerce")


for col in climate_data_2007.columns:
    if col != "Data" and climate_data_2007[col].dtype == object:
        climate_data_2007[col] = (
            climate_data_2007[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2007[col] = pd.to_numeric(climate_data_2007[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2007 = climate_data_2007.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2007.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2007.head()

0   2007-01-01
1   2007-01-01
2   2007-01-01
3   2007-01-01
4   2007-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2007-01-01,74.0,386.257143,18.450000,21.7,16.4,17.300000,93.041667,10.1,1.504167
1,2007-01-02,6.8,1134.250000,20.650000,25.0,17.7,18.033333,85.750000,5.9,1.612500
2,2007-01-03,4.8,915.321429,20.866667,26.3,18.6,18.329167,86.041667,9.0,1.791667
3,2007-01-04,0.0,1633.192857,20.925000,26.3,17.9,16.954167,79.541667,10.3,2.395833
4,2007-01-05,1.8,1067.984615,21.950000,27.4,19.1,16.883333,73.916667,9.6,1.895833


In [12]:
climate_data_2008 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2008.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2008-01-01,00:00,0,"885,1","885,1","884,1",-9999,"23,9","11,6","24,6","23,8","11,7","10,9",46,42,46,87,6,2,NaN
1,2008-01-01,01:00,0,"885,4","885,5","885,1",-9999,"21,3","12,4","23,9","20,7","13,3","11,6",62,46,57,44,"3,4","1,1",NaN
2,2008-01-01,02:00,0,"885,5","885,5","885,3",-9999,"19,7","12,7","21,9","19,6","13,4","12,1",64,55,64,29,2,0,NaN
3,2008-01-01,03:00,0,"885,2","885,5","885,2",-9999,"21,1","12,8","22,5","19,7","13,1","12,5",66,54,59,68,"3,1","1,3",NaN
4,2008-01-01,04:00,0,"884,8","885,2","884,8",-9999,"19,5",13,"21,1","18,3","13,3","12,8",72,60,66,23,"2,6",",9",NaN


In [13]:
climate_data_2008['DATA (YYYY-MM-DD)'] = climate_data_2008['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2008['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2008['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2008['DATA (YYYY-MM-DD)'].head(), climate_data_2008['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2008.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2008.columns:
    if col != "Data":
        climate_data_2008[col] = (
            climate_data_2008[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2008[col] = pd.to_numeric(climate_data_2008[col], errors="coerce")


for col in climate_data_2008.columns:
    if col != "Data" and climate_data_2008[col].dtype == object:
        climate_data_2008[col] = (
            climate_data_2008[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2008[col] = pd.to_numeric(climate_data_2008[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2008 = climate_data_2008.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2008.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2008.head()

0   2008-01-01
1   2008-01-01
2   2008-01-01
3   2008-01-01
4   2008-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2008-01-01,0.0,1888.164286,23.916667,30.1,16.4,11.679167,49.041667,7.5,1.670833
1,2008-01-02,0.0,1798.871429,24.300000,29.6,18.4,13.537500,52.250000,7.5,2.070833
2,2008-01-03,0.0,1575.471429,23.154167,28.3,18.6,15.445833,64.000000,7.5,1.891667
3,2008-01-04,0.0,1963.671429,24.175000,29.1,19.8,13.437500,52.208333,8.8,2.641667
4,2008-01-05,0.0,1831.078571,24.008333,30.3,17.4,12.633333,50.666667,8.0,1.875000


In [14]:
climate_data_2009 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2009_A_31-12-2009.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2009.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2009-01-01,00:00,0,"885,4","885,4","884,7",-9999,"22,4","19,6","22,9","22,2",20,"18,7",86,78,84,22,"1,7",",8",NaN
1,2009-01-01,01:00,0,"885,8","885,9","885,4",-9999,"23,2",19,"23,2","21,7","20,2","18,8",86,77,77,133,"1,9","1,3",NaN
2,2009-01-01,02:00,0,"886,4","886,5","885,8",-9999,"22,4","19,4","23,3","21,6","19,8","19,1",87,77,83,357,"1,9",",9",NaN
3,2009-01-01,03:00,0,"886,4","886,8","886,4",-9999,"21,9","19,3","22,5","21,2","19,6","18,9",87,82,85,177,"1,8","1,1",NaN
4,2009-01-01,04:00,0,"886,1","886,5","886,1",-9999,"22,3","18,4","22,5","21,7","19,3","17,9",86,77,79,189,"2,8","1,3",NaN


In [15]:
climate_data_2009['DATA (YYYY-MM-DD)'] = climate_data_2009['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2009['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2009['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2009['DATA (YYYY-MM-DD)'].head(), climate_data_2009['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2009.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2009.columns:
    if col != "Data":
        climate_data_2009[col] = (
            climate_data_2009[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2009[col] = pd.to_numeric(climate_data_2009[col], errors="coerce")


for col in climate_data_2009.columns:
    if col != "Data" and climate_data_2009[col].dtype == object:
        climate_data_2009[col] = (
            climate_data_2009[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2009[col] = pd.to_numeric(climate_data_2009[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2009 = climate_data_2009.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2009.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2009.head()

0   2009-01-01
1   2009-01-01
2   2009-01-01
3   2009-01-01
4   2009-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2009-01-01,21.8,1736.800000,23.229167,29.9,18.7,17.795833,73.708333,15.0,1.904167
1,2009-01-02,0.8,1467.118750,23.770833,30.4,18.8,17.616667,70.916667,8.3,2.170833
2,2009-01-03,2.2,1364.564286,22.395833,26.7,19.4,18.279167,78.000000,10.3,2.783333
3,2009-01-04,0.0,1733.707143,21.858333,26.5,18.9,17.883333,79.250000,11.4,3.300000
4,2009-01-05,16.2,835.007143,19.945833,24.0,17.8,17.658333,87.041667,9.3,3.016667


In [16]:
climate_data_2010 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2010_A_31-12-2010.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2010.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2010-01-01,00:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN
1,2010-01-01,01:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN
2,2010-01-01,02:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN
3,2010-01-01,03:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN
4,2010-01-01,04:00,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,NaN


In [17]:
climate_data_2010['DATA (YYYY-MM-DD)'] = climate_data_2010['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2010['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2010['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2010['DATA (YYYY-MM-DD)'].head(), climate_data_2010['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2010.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2010.columns:
    if col != "Data":
        climate_data_2010[col] = (
            climate_data_2010[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2010[col] = pd.to_numeric(climate_data_2010[col], errors="coerce")


for col in climate_data_2010.columns:
    if col != "Data" and climate_data_2010[col].dtype == object:
        climate_data_2010[col] = (
            climate_data_2010[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2010[col] = pd.to_numeric(climate_data_2010[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2010 = climate_data_2010.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2010.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2010.head()

0   2010-01-01
1   2010-01-01
2   2010-01-01
3   2010-01-01
4   2010-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2010-01-01,6.6,1203.130769,22.035714,26.0,18.9,18.821429,82.500000,7.9,2.592857
1,2010-01-02,6.6,1608.721429,21.333333,26.5,18.2,17.320833,79.541667,11.4,3.737500
2,2010-01-03,0.2,2101.342857,22.829167,28.1,18.0,17.150000,72.541667,9.3,2.887500
3,2010-01-04,0.0,1954.107143,23.900000,29.2,19.7,15.795833,61.250000,9.0,3.066667
4,2010-01-05,0.0,1993.100000,24.391667,29.5,17.6,15.625000,60.416667,8.1,1.895833


In [18]:
climate_data_2011 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2011_A_31-12-2011.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2011.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2011-01-01,00:00,0,"881,8","881,8",881,-9999,"19,2","18,3","19,4","19,2","18,5","18,3",95,94,95,342,"5,8","3,4",NaN
1,2011-01-01,01:00,0,"882,4","882,4","881,7",-9999,"19,3","18,2","19,3","19,2","18,4","18,2",95,93,93,334,"5,6","2,2",NaN
2,2011-01-01,02:00,0,"882,7","882,7","882,4",-9999,"19,1","18,3","19,3","19,1","18,3","18,2",95,93,95,347,"5,6",3,NaN
3,2011-01-01,03:00,6,"882,6","882,9","882,6",",3","18,1","17,6","19,1","18,1","18,4","17,6",97,95,96,310,"7,9","3,7",NaN
4,2011-01-01,04:00,3,882,"882,6",882,-9999,"17,7","17,2","18,2","17,7","17,6","17,1",97,96,96,335,"6,7","2,4",NaN


In [19]:
climate_data_2011['DATA (YYYY-MM-DD)'] = climate_data_2011['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2011['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2011['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2011['DATA (YYYY-MM-DD)'].head(), climate_data_2011['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2011.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2011.columns:
    if col != "Data":
        climate_data_2011[col] = (
            climate_data_2011[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2011[col] = pd.to_numeric(climate_data_2011[col], errors="coerce")


for col in climate_data_2011.columns:
    if col != "Data" and climate_data_2011[col].dtype == object:
        climate_data_2011[col] = (
            climate_data_2011[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2011[col] = pd.to_numeric(climate_data_2011[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2011 = climate_data_2011.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2011.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2011.head()

0   2011-01-01
1   2011-01-01
2   2011-01-01
3   2011-01-01
4   2011-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2011-01-01,26.4,1010.580000,19.558333,24.6,17.7,18.008333,91.000000,8.9,3.225000
1,2011-01-02,14.4,431.035714,18.525000,19.3,17.9,17.583333,94.333333,12.6,4.579167
2,2011-01-03,13.0,424.157143,18.437500,19.8,17.5,17.454167,94.041667,12.0,4.987500
3,2011-01-04,14.6,932.457143,19.404167,23.1,18.0,18.133333,92.625000,9.7,3.395833
4,2011-01-05,3.2,1413.471429,20.841667,27.1,17.9,18.004167,84.833333,11.1,2.308333


In [20]:
climate_data_2012 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2012_A_31-12-2012.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2012.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2012-01-01,00:00,0,"884,7","884,7","883,8",-9999,"19,9","17,5","19,9","19,4",18,"17,5",91,86,86,322,"5,7",3,NaN
1,2012-01-01,01:00,0,"885,2","885,3","884,7",-9999,"19,7","17,9","19,9","19,7","17,9","17,5",89,86,89,331,"5,9","2,5",NaN
2,2012-01-01,02:00,0,"885,5","885,5","885,2",-9999,"19,7","17,9","19,8","19,6",18,"17,9",90,89,90,321,"5,6",3,NaN
3,2012-01-01,03:00,0,"884,9","885,5","884,9",-9999,"19,6","16,9","19,7","19,4","17,9","16,9",90,85,85,313,"5,7","3,4",NaN
4,2012-01-01,04:00,0,"884,3","884,9","884,3",-9999,"19,1","17,1","19,6","19,1","17,1","16,8",88,84,88,326,"6,9","3,2",NaN


In [21]:
climate_data_2012['DATA (YYYY-MM-DD)'] = climate_data_2012['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2012['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2012['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2012['DATA (YYYY-MM-DD)'].head(), climate_data_2012['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2012.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2012.columns:
    if col != "Data":
        climate_data_2012[col] = (
            climate_data_2012[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2012[col] = pd.to_numeric(climate_data_2012[col], errors="coerce")


for col in climate_data_2012.columns:
    if col != "Data" and climate_data_2012[col].dtype == object:
        climate_data_2012[col] = (
            climate_data_2012[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2012[col] = pd.to_numeric(climate_data_2012[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2012 = climate_data_2012.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2012.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2012.head()

0   2012-01-01
1   2012-01-01
2   2012-01-01
3   2012-01-01
4   2012-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2012-01-01,5.0,1336.157143,20.462500,25.2,17.9,17.879167,85.625000,11.9,3.800000
1,2012-01-02,21.6,459.123077,19.191667,20.8,18.3,18.004167,93.000000,16.0,4.404167
2,2012-01-03,13.0,779.807143,19.429167,24.1,17.8,17.137500,87.333333,8.3,2.139130
3,2012-01-04,0.4,1572.621429,20.166667,25.9,15.6,16.300000,80.125000,8.0,2.529167
4,2012-01-05,0.0,1786.171429,20.654167,26.5,16.5,16.383333,77.833333,9.0,3.183333


In [22]:
climate_data_2013 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2013_A_31-12-2013.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2013.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2013-01-01,00:00,0,"885,7","885,7","885,1",-9999,"24,1","14,2",25,"24,1","14,4","13,9",54,50,54,112,"4,6","2,3",NaN
1,2013-01-01,01:00,0,"886,5","886,5","885,7",-9999,"23,5","14,5","24,2","23,4","14,6","14,2",58,54,57,110,5,"2,7",NaN
2,2013-01-01,02:00,0,"886,9","886,9","886,5",-9999,"23,3","14,5","23,5","23,2","14,7","14,4",59,56,58,91,"5,4","3,4",NaN
3,2013-01-01,03:00,0,"886,5","886,9","886,5",-9999,"22,8","14,6","23,3","22,8","14,7","14,4",60,57,60,81,"5,4","3,1",NaN
4,2013-01-01,04:00,0,886,"886,5",886,-9999,"22,4","14,5","22,9","22,2","14,6","14,3",62,59,61,86,"5,3","2,9",NaN


In [23]:
climate_data_2013['DATA (YYYY-MM-DD)'] = climate_data_2013['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2013['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2013['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2013['DATA (YYYY-MM-DD)'].head(), climate_data_2013['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2013.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2013.columns:
    if col != "Data":
        climate_data_2013[col] = (
            climate_data_2013[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2013[col] = pd.to_numeric(climate_data_2013[col], errors="coerce")


for col in climate_data_2013.columns:
    if col != "Data" and climate_data_2013[col].dtype == object:
        climate_data_2013[col] = (
            climate_data_2013[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2013[col] = pd.to_numeric(climate_data_2013[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2013 = climate_data_2013.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2013.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2013.head()

0   2013-01-01
1   2013-01-01
2   2013-01-01
3   2013-01-01
4   2013-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2013-01-01,0.0,1781.457143,24.708333,29.6,18.1,13.266667,50.416667,7.5,2.329167
1,2013-01-02,0.0,1945.707143,24.208333,30.1,18.2,12.466667,51.666667,7.7,1.883333
2,2013-01-03,0.0,2023.971429,24.554167,30.6,17.1,12.666667,50.000000,7.9,2.195833
3,2013-01-04,0.2,1666.792857,23.495833,29.0,17.7,15.783333,64.250000,10.8,2.358333
4,2013-01-05,0.2,2047.585714,23.991667,29.2,19.0,15.308333,61.541667,7.5,2.112500


In [24]:
climate_data_2014 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2014_A_31-12-2014.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2014.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2014-01-01,00:00,0,"885,8","885,9","885,4",-9999,"21,2",18,"22,2",21,"18,3","17,6",83,77,82,173,"2,1",",8",NaN
1,2014-01-01,01:00,0,"886,6","886,6","885,8",-9999,"20,3","18,9","21,2","20,1",19,"17,7",92,82,91,167,"1,4",",4",NaN
2,2014-01-01,02:00,0,887,887,"886,6",-9999,"19,9","18,5","20,4","19,7","18,8","18,2",93,90,91,187,"1,4",",6",NaN
3,2014-01-01,03:00,0,"886,5",887,"886,5",-9999,"19,9","18,2","20,1","19,6","18,5",18,92,88,90,120,"1,2",",5",NaN
4,2014-01-01,04:00,0,"886,2","886,5","886,2",-9999,"19,4","18,5","20,1","19,1","18,8","17,9",96,88,95,124,",6",",3",NaN


In [25]:
climate_data_2014['DATA (YYYY-MM-DD)'] = climate_data_2014['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2014['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2014['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2014['DATA (YYYY-MM-DD)'].head(), climate_data_2014['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2014.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2014.columns:
    if col != "Data":
        climate_data_2014[col] = (
            climate_data_2014[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2014[col] = pd.to_numeric(climate_data_2014[col], errors="coerce")


for col in climate_data_2014.columns:
    if col != "Data" and climate_data_2014[col].dtype == object:
        climate_data_2014[col] = (
            climate_data_2014[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2014[col] = pd.to_numeric(climate_data_2014[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2014 = climate_data_2014.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2014.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2014.head()

0   2014-01-01
1   2014-01-01
2   2014-01-01
3   2014-01-01
4   2014-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2014-01-01,0.0,1550.135714,22.954167,29.2,18.3,17.558333,74.541667,6.9,1.558333
1,2014-01-02,0.0,2025.292857,23.633333,30.1,18.2,16.666667,68.291667,7.6,2.095833
2,2014-01-03,0.0,2164.378571,24.291667,29.7,17.4,14.408333,58.500000,7.9,2.000000
3,2014-01-04,0.0,2103.221429,24.608333,29.8,17.7,11.758333,46.583333,11.5,2.704167
4,2014-01-05,0.0,2262.921429,23.837500,29.3,16.7,11.866667,49.333333,9.0,2.566667


In [26]:
climate_data_2015 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2015_A_31-12-2015.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2015.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2015-01-01,00:00,0,"886,8","886,8","886,2",-9999,"20,1","15,7","20,9",20,"16,6","15,4",80,72,76,84,1,",5",NaN
1,2015-01-01,01:00,0,"887,5","887,6","886,8",-9999,"19,8",16,"20,1","19,2",16,"15,4",80,76,79,20,"1,1",",4",NaN
2,2015-01-01,02:00,0,"888,1","888,1","887,5",-9999,"18,6","15,7",20,"18,5","16,3","15,5",84,78,84,284,"1,5",1,NaN
3,2015-01-01,03:00,0,"887,8","888,2","887,8",-9999,"18,6","15,9","19,5","18,6","16,1","15,6",84,80,84,265,"1,3",",7",NaN
4,2015-01-01,04:00,0,"887,1","887,9","887,1",-9999,"19,3",16,"19,5","18,6","16,4","15,9",84,81,81,110,",9",",2",NaN


In [27]:
climate_data_2015['DATA (YYYY-MM-DD)'] = climate_data_2015['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2015['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2015['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2015['DATA (YYYY-MM-DD)'].head(), climate_data_2015['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2015.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2015.columns:
    if col != "Data":
        climate_data_2015[col] = (
            climate_data_2015[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2015[col] = pd.to_numeric(climate_data_2015[col], errors="coerce")


for col in climate_data_2015.columns:
    if col != "Data" and climate_data_2015[col].dtype == object:
        climate_data_2015[col] = (
            climate_data_2015[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2015[col] = pd.to_numeric(climate_data_2015[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2015 = climate_data_2015.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2015.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2015.head()

0   2015-01-01
1   2015-01-01
2   2015-01-01
3   2015-01-01
4   2015-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2015-01-01,0.0,1905.607143,23.491667,30.4,16.9,14.483333,61.125000,8.4,1.775000
1,2015-01-02,0.0,1143.485714,23.250000,30.2,17.8,15.354167,62.833333,7.4,1.508333
2,2015-01-03,3.4,1544.285714,23.033333,30.5,18.8,17.120833,71.000000,10.6,1.916667
3,2015-01-04,10.2,1611.746667,22.162500,27.3,17.2,17.025000,74.166667,12.2,3.029167
4,2015-01-05,0.0,1963.114286,22.750000,29.0,16.1,14.241667,64.166667,7.9,2.095833


In [28]:
climate_data_2016 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2016_A_31-12-2016.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2016.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2016-01-01,00:00,0,"886,6","886,6",886,-9999,"21,5","17,4","23,3","21,5","17,9",17,79,69,78,348,"2,5",",4",NaN
1,2016-01-01,01:00,0,"887,5","887,5","886,6",-9999,"21,5",17,"21,9","21,2","18,3",17,82,75,76,327,"2,3","1,1",NaN
2,2016-01-01,02:00,0,"887,3","887,6","887,3",-9999,"21,4","17,2","21,6","21,3","17,4",17,79,75,77,276,"2,4","1,3",NaN
3,2016-01-01,03:00,0,"886,8","887,4","886,8",-9999,"21,3","16,9","21,5","21,3","17,2","16,9",77,76,76,276,"3,1","1,5",NaN
4,2016-01-01,04:00,0,"886,3","886,8","886,3",-9999,"20,9","17,4","21,3","20,5","17,4","16,8",82,76,80,284,"2,6","1,3",NaN


In [29]:
climate_data_2016['DATA (YYYY-MM-DD)'] = climate_data_2016['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2016['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2016['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2016['DATA (YYYY-MM-DD)'].head(), climate_data_2016['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2016.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2016.columns:
    if col != "Data":
        climate_data_2016[col] = (
            climate_data_2016[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2016[col] = pd.to_numeric(climate_data_2016[col], errors="coerce")


for col in climate_data_2016.columns:
    if col != "Data" and climate_data_2016[col].dtype == object:
        climate_data_2016[col] = (
            climate_data_2016[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2016[col] = pd.to_numeric(climate_data_2016[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2016 = climate_data_2016.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2016.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2016.head()

0   2016-01-01
1   2016-01-01
2   2016-01-01
3   2016-01-01
4   2016-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2016-01-01,1.8,1567.578571,23.720833,28.9,19.0,16.895833,68.125000,7.1,1.520833
1,2016-01-02,3.6,1650.607143,22.383333,29.0,18.1,17.862500,77.875000,8.7,1.858333
2,2016-01-03,32.6,675.507143,21.750000,26.0,18.1,18.408333,82.208333,14.5,1.908333
3,2016-01-04,0.2,1088.100000,21.425000,27.7,18.8,18.820833,86.750000,11.9,1.366667
4,2016-01-05,6.4,780.985714,20.829167,25.3,18.1,19.500000,92.750000,8.4,1.516667


In [30]:
climate_data_2017 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\INMET_CO_DF_A001_BRASILIA_01-01-2017_A_31-12-2017.CSV",
                                skiprows=8,
                                sep=";",         
                                encoding="latin1")

climate_data_2017.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2017-01-01,00:00,0,"887,6","887,6",887,-9999,22,"13,9","23,2","21,8",14,"12,8",61,52,60,55,"4,2","1,5",NaN
1,2017-01-01,01:00,0,"887,8","887,8","887,5",-9999,"18,4","12,5",22,"18,4","13,9","12,3",72,59,68,61,3,",9",NaN
2,2017-01-01,02:00,0,"888,1","888,1","887,8",-9999,"17,7","11,6","18,4","17,1","13,3","10,4",74,63,67,186,"2,1","1,3",NaN
3,2017-01-01,03:00,0,"887,5","888,1","887,5",-9999,"17,9","11,7","18,6","17,4","12,5","10,7",73,61,67,146,"1,6",",7",NaN
4,2017-01-01,04:00,0,887,"887,5",887,-9999,"17,1","10,8","18,5",17,"12,6","10,3",74,63,67,145,2,",9",NaN


In [31]:
climate_data_2017['DATA (YYYY-MM-DD)'] = climate_data_2017['DATA (YYYY-MM-DD)'].str.strip()

climate_data_2017['DATA (YYYY-MM-DD)'] = pd.to_datetime(climate_data_2017['DATA (YYYY-MM-DD)'], format="%Y-%m-%d", errors='coerce')

print(climate_data_2017['DATA (YYYY-MM-DD)'].head(), climate_data_2017['DATA (YYYY-MM-DD)'].isna().sum())

import numpy as np

climate_data_2017.rename(columns={"DATA (YYYY-MM-DD)": "Data"}, inplace=True)

for col in climate_data_2017.columns:
    if col != "Data":
        climate_data_2017[col] = (
            climate_data_2017[col].astype(str)
            .str.replace(",", ".", regex=False)
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2017[col] = pd.to_numeric(climate_data_2017[col], errors="coerce")


for col in climate_data_2017.columns:
    if col != "Data" and climate_data_2017[col].dtype == object:
        climate_data_2017[col] = (
            climate_data_2017[col].astype(str)
            .str.replace(",", ".")
            .str.strip()
            .replace("-9999", np.nan)
            .replace("", np.nan)
        )
        climate_data_2017[col] = pd.to_numeric(climate_data_2017[col], errors="coerce")

# 5. Agregar por dia
meteo_daily_2017 = climate_data_2017.groupby("Data").agg({
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "RADIACAO GLOBAL (KJ/m²)": "mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "mean",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "mean",
    "VENTO, RAJADA MAXIMA (m/s)": "max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "mean"
}).reset_index()

# 6. Renomear colunas
meteo_daily_2017.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao_mm",
    "RADIACAO GLOBAL (KJ/m²)": "radiacao_media",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temp_media",
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "temp_max",
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "temp_min",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "temp_orvalho_media",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "humidade_media",
    "VENTO, RAJADA MAXIMA (m/s)": "vento_rajada_max",
    "VENTO, VELOCIDADE HORARIA (m/s)": "vento_vel_media"
}, inplace=True)

meteo_daily_2017.head()

0   2017-01-01
1   2017-01-01
2   2017-01-01
3   2017-01-01
4   2017-01-01
Name: DATA (YYYY-MM-DD), dtype: datetime64[ns] 0


,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2017-01-01,0.0,1658.771429,22.870833,29.5,16.2,10.675000,49.375000,8.1,1.279167
1,2017-01-02,0.0,1567.184615,23.500000,29.7,17.0,12.808333,53.833333,5.1,1.183333
2,2017-01-03,10.4,1483.084615,23.837500,29.9,19.0,15.437500,60.458333,17.8,1.704167
3,2017-01-04,9.4,1187.815385,21.654167,26.9,18.3,16.795833,74.500000,11.5,2.041667
4,2017-01-05,1.2,1930.521429,22.445833,29.5,17.7,15.895833,69.208333,7.8,2.095833


In [32]:
merged_meteo_03_17 = pd.concat([
    meteo_daily_2003,meteo_daily_2004, meteo_daily_2005, meteo_daily_2006, meteo_daily_2007, meteo_daily_2008, meteo_daily_2009, meteo_daily_2010, meteo_daily_2011, meteo_daily_2012, meteo_daily_2013, meteo_daily_2014, meteo_daily_2015, meteo_daily_2016, meteo_daily_2017], ignore_index=True)

merged_meteo_03_17.head()

,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2003-01-01,5.4,1123.000000,21.091667,26.0,18.8,18.250000,84.500000,10.1,2.395833
1,2003-01-02,0.6,1652.214286,22.233333,27.2,18.9,18.033333,78.583333,10.9,2.741667
2,2003-01-03,8.4,1365.285714,21.658333,26.6,18.3,18.266667,81.708333,11.6,3.562500
3,2003-01-04,25.4,1423.785714,21.350000,26.8,17.5,17.716667,81.250000,13.9,4.216667
4,2003-01-05,0.8,1469.846154,21.879167,27.1,17.4,17.162500,75.875000,10.8,4.137500


In [33]:
merged_meteo_03_17.to_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\merged_meteo_03_17.csv", index=False) 

In [34]:
merged_meteo_03_17.shape

(5479, 10)

In [35]:
import pandas as pd

In [36]:
merged_meteo_18_24 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Climate_data\merged_meteo.csv")
merged_meteo_18_24.head()

,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2024-01-01,35.6,1094.385714,21.995833,26.7,19.4,18.666667,82.541667,6.2,1.658333
1,2024-01-02,23.2,1171.950000,21.187500,26.6,19.3,19.570833,91.000000,6.7,1.787500
2,2024-01-03,48.8,724.321429,20.537500,23.4,19.1,19.370833,93.375000,8.2,2.250000
3,2024-01-04,7.6,590.893333,20.375000,23.3,19.3,19.491667,94.750000,8.4,2.529167
4,2024-01-05,2.4,833.607143,20.400000,23.2,18.9,19.004167,92.125000,11.1,2.545833


In [37]:
merged_meteo_03_17["Data"] = pd.to_datetime(merged_meteo_03_17["Data"])
merged_meteo_18_24["Data"] = pd.to_datetime(merged_meteo_18_24["Data"])

In [38]:
df_total = pd.concat([merged_meteo_03_17, merged_meteo_18_24], ignore_index=True)

# Order by Date
df_total = df_total.sort_values("Data").reset_index(drop=True)

# Save the final DataFrame to a CSV file
df_total.to_csv("meteo_2003_2024.csv", index=False)

print(df_total.head())
print(df_total.tail())

        Data  precipitacao_mm  radiacao_media  temp_media  temp_max  temp_min  \
0 2003-01-01              5.4     1123.000000   21.091667      26.0      18.8   
1 2003-01-02              0.6     1652.214286   22.233333      27.2      18.9   
2 2003-01-03              8.4     1365.285714   21.658333      26.6      18.3   
3 2003-01-04             25.4     1423.785714   21.350000      26.8      17.5   
4 2003-01-05              0.8     1469.846154   21.879167      27.1      17.4   

   temp_orvalho_media  humidade_media  vento_rajada_max  vento_vel_media  
0           18.250000       84.500000              10.1         2.395833  
1           18.033333       78.583333              10.9         2.741667  
2           18.266667       81.708333              11.6         3.562500  
3           17.716667       81.250000              13.9         4.216667  
4           17.162500       75.875000              10.8         4.137500  
           Data  precipitacao_mm  radiacao_media  temp_media  t

In [39]:
df_total.shape

(8036, 10)